<a href="https://colab.research.google.com/github/mateopolancecofficial/NLP/blob/main/WineReviews/SequenceTextModelWithPretrainedEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q -U tensorflow-text

     |████████████████████████████████| 3.4MB 23.7MB/s 


In [2]:
pip install -q -U tf-models-official

     |████████████████████████████████| 1.1MB 17.8MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 102kB 13.3MB/s 
     |████████████████████████████████| 706kB 51.0MB/s 
     |████████████████████████████████| 358kB 55.5MB/s 
     |████████████████████████████████| 37.6MB 79kB/s 
     |████████████████████████████████| 1.2MB 56.5MB/s 
     |████████████████████████████████| 645kB 55.0MB/s 
     |████████████████████████████████| 174kB 56.6MB/s 


In [3]:
pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 7.8MB/s 


In [4]:
!git clone -l -s https://github.com/mateopolancecofficial/NLP.git

Cloning into 'NLP'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 72 (delta 34), reused 25 (delta 6), pack-reused 0
Unpacking objects: 100% (72/72), done.


In [12]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.layers import (
    Dense,
    Dropout,
)
import kerastuner as kt
from official.nlp import optimization
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

tf.get_logger().setLevel('ERROR')
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [9]:
import os

if tf.test.is_gpu_available():
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')

Using GPU


In [45]:
# set parameters
AUTOTUNE = tf.data.AUTOTUNE
path_v1 = "/content/NLP/WineReviews/data/winemag-data-130k-v2.csv" 
path_v2 = "/content/NLP/WineReviews/data/winemag-data_first150k.csv"
checkpoint_path = "/content/NLP/WineReviews/checkpoints/cp-{epoch:04d}.ckpt"
batch_size = 1024
seed = 42
col_idx = 0
train_size, test_size, val_size = 0.8, 0.2, 0.2
transform = 'normalize'
columns = ['description', 'points']

NameError: ignored

In [11]:
def load_data(path_v1: str, path_v2: str, columns: list, col_idx: int):
  """
  Load and concatenate two datasets with removing duplicates.
  param path_v1: import path of first dataset
  param path_v2: import path of second dataset
  param columns: list of columns to preserve in dataframe
  param col_idx: index of column given in input columns list 
                 on which look for duplicates in dataframe
  return:        pandas dataframe
  """
  
  df_v1 = pd.read_csv(path_v1, index_col=0)
  df_v1 = df_v1[columns]
  # remove numbers form column description from first dataframe
  df_v1.description = df_v1.description.str.replace('\d+', '')

  df_v2 = pd.read_csv(path_v2, index_col=0)
  df_v2 = df_v2[columns]
  # remove numbers form column description from second dataframe
  df_v2.description = df_v2.description.str.replace('\d+', '')

  df = pd.concat([df_v1, df_v2])

  # dropping duplicte values
  df.drop_duplicates(subset = columns[col_idx],
                       keep = 'first', inplace = True)
  
  return df

In [41]:
def split_data(df: pd.DataFrame, train_size: float, test_size: float, 
               val_size: float):
  """
  Split dataset on train, test and validation subsets.
  param df:          input dataframe
  param train_size:  fraction of train size
  param test_size:   fraction of test size
  param val_size:    fraction of validation size
  return:            dictionary, keys=names of dataframes, columns=dataframes
  """
  
  # shuffle dataset
  df = df.sample(frac = 1)
  
  # split on test and train set
  text_train, text_test, y_train, y_test = train_test_split(df.description, df.points,
                                           test_size=test_size, train_size=train_size)
  
  y_train, y_test = y_train.astype('float'), y_test.astype('float')
  
  # split train set on train and validation subsets
  text_train, text_val, y_train, y_val = train_test_split(text_train, y_train,
                                                  test_size=val_size, train_size=train_size)
  
  y_train, y_val = y_train.astype('float'), y_val.astype('float')

  y_train = y_train.to_numpy().reshape(-1, 1)
  y_val = y_val.to_numpy().reshape(-1, 1)
  y_test = y_test.to_numpy().reshape(-1, 1)

 # apply target variable transformation
    if transform == 'normalize':
      scaler = MinMaxScaler()
      scaler.fit(y_train)
      y_train = scaler.transform(y_train)
      y_val = scaler.transform(y_val)
      y_test = scaler.transform(y_test)
    
    elif transform == 'standardize':
      scaler = StandardScaler()
      scaler.fit(y_train)
      y_train = scaler.transform(y_train)
      y_val = scaler.transform(y_val)
      y_test = scaler.transform(y_test)
    
    else:
      scaler = MinMaxScaler()
      scaler.fit(y_train)
      y_train = scaler.transform(y_train)
      y_val = scaler.transform(y_val)
      y_test = scaler.transform(y_test)
  
  y_train, y_val, y_test = y_train.ravel(), y_val.ravel(), y_test.ravel()

  dataset_dict = {
      'text_train': text_train,
      'y_train': y_train,
      'text_val': text_val,
      'y_val': y_val,
      'text_test': text_test,
      'y_test': y_test,
      'scaler': scaler
  }

  return dataset_dict

In [19]:
def create_input_datasets(df_data: dict):
  """
  Create tensorflow datasets based on input dataframes for train, validation 
  and test subsets.
  param df_data: dictionary, keys=names of dataframes, columns=dataframes
  return:        dictionary, keys=names of datasets, columns=datasets
  """

  # create train dataset for input in tensorflow model
  train_dataset = tf.data.Dataset.from_tensor_slices((df_data['text_train'], 
                                                      df_data['y_train']))
  train_dataset = train_dataset.batch(batch_size)
  train_ds = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)

  # create validation dataset for input in tensorflow model
  val_dataset = tf.data.Dataset.from_tensor_slices((df_data['text_val'], 
                                                    df_data['y_val']))
  val_dataset = val_dataset.batch(batch_size)
  val_ds = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

  # create validation dataset for input in tensorflow model
  test_dataset = tf.data.Dataset.from_tensor_slices((df_data['text_test'], 
                                                     df_data['y_test']))
  test_dataset = test_dataset.batch(batch_size)
  test_ds = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

  datasets = {
      'train_dataset': train_dataset,
      'val_dataset': val_dataset,
      'test_dataset': test_dataset
  } 

  return datasets

In [20]:
# define loss functions
from tensorflow.keras import backend as K

def rmse():
  def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
  return root_mean_squared_error

def rmsle():
  def root_mean_squared_log_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(K.log(1+y_pred) - K.log(1+y_true))))
  return root_mean_squared_log_error

In [21]:
# disable eager execution
tf.config.run_functions_eagerly(False)

def model_builder(hp):
  # use pretrained embeddings for input layer
  hub_model = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
  # 'trainable=True' - boolean controlling whether this layer is trainable
  hub_layer = hub.KerasLayer(hub_model, input_shape=[], dtype=tf.string, 
                             trainable=True)
  model = tf.keras.Sequential()
  model.add(hub_layer)
  hp_units_1 = hp.Int('units_1', min_value=64, max_value=128, step=16)
  activation=hp.Choice(
        'dense_activation',
        values=['relu', 'tanh', 'sigmoid'],
        default='relu'
    )
  model.add(Dense(units=hp_units_1, activation=activation))
  model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.1,
                max_value=0.5,
                default=0.25,
                step=0.1,
            ))
        )
  hp_units_2 = hp.Int('units_2', min_value=8, max_value=64, step=16)
  model.add(Dense(units=hp_units_2, activation=activation))
  model.add(
            Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.1,
                max_value=0.5,
                default=0.25,
                step=0.1,
            ))
        )
  model.add(Dense(1))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
              loss=rmse(),
              metrics=['mean_absolute_error'])

  return model


In [22]:
# call data transformation functions
df = load_data(path_v1, path_v2, columns, col_idx)
df_data = split_data(df, train_size, test_size, val_size)


In [26]:
datasets = create_input_datasets(df_data)

In [30]:
tuner = kt.Hyperband(model_builder,
                     objective='mean_absolute_error',
                     max_epochs=5,
                     directory='SequenceTextPretrained'
                    )
                    

In [38]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

In [31]:

tuner.search(df_data['text_train'], df_data['y_train'], 
             validation_data=(df_data['text_val'], df_data['y_val']), 
             epochs=5, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


Trial 10 Complete [00h 25m 54s]
mean_absolute_error: 0.3790609538555145

Best mean_absolute_error So Far: 0.3703858256340027
Total elapsed time: 02h 40m 27s


In [32]:
model = tuner.hypermodel.build(best_hps)

In [39]:
# checkpoints callback is not defined because a lack of disk space on Google Colab
history = model.fit(datasets['train_dataset'],
                    validation_data=datasets['val_dataset'],
                    epochs=100,
                    callbacks=[stop_early]
                    )

Epoch 1/100
135/135 [==============================] - 11s 81ms/step - loss: 0.3587 - mean_absolute_error: 0.2776 - val_loss: 0.5575 - val_mean_absolute_error: 0.4355
Epoch 2/100
135/135 [==============================] - 11s 82ms/step - loss: 0.3546 - mean_absolute_error: 0.2743 - val_loss: 0.5460 - val_mean_absolute_error: 0.4282
Epoch 3/100
135/135 [==============================] - 11s 83ms/step - loss: 0.3549 - mean_absolute_error: 0.2758 - val_loss: 0.5520 - val_mean_absolute_error: 0.4322
Epoch 4/100
135/135 [==============================] - 11s 82ms/step - loss: 0.3388 - mean_absolute_error: 0.2617 - val_loss: 0.5523 - val_mean_absolute_error: 0.4318
Epoch 5/100
135/135 [==============================] - 11s 82ms/step - loss: 0.3261 - mean_absolute_error: 0.2512 - val_loss: 0.5534 - val_mean_absolute_error: 0.4325
Epoch 6/100
135/135 [==============================] - 11s 81ms/step - loss: 0.3219 - mean_absolute_error: 0.2483 - val_loss: 0.5651 - val_mean_absolute_error: 0.443

In [43]:
df_data = split_data(df, train_size, test_size, val_size)
datasets = create_input_datasets(df_data)

In [44]:
history = model.fit(datasets['train_dataset'],
                    validation_data=datasets['val_dataset'],
                    epochs=100,
                    callbacks=[stop_early]
                    )

Epoch 1/100
106/106 [==============================] - 9s 84ms/step - loss: 0.1776 - mean_absolute_error: 0.1420 - val_loss: 0.0749 - val_mean_absolute_error: 0.0588
Epoch 2/100
106/106 [==============================] - 9s 84ms/step - loss: 0.0940 - mean_absolute_error: 0.0734 - val_loss: 0.0689 - val_mean_absolute_error: 0.0536
Epoch 3/100
106/106 [==============================] - 9s 83ms/step - loss: 0.0838 - mean_absolute_error: 0.0652 - val_loss: 0.0663 - val_mean_absolute_error: 0.0510
Epoch 4/100
106/106 [==============================] - 9s 84ms/step - loss: 0.0773 - mean_absolute_error: 0.0599 - val_loss: 0.0632 - val_mean_absolute_error: 0.0482
Epoch 5/100
106/106 [==============================] - 9s 83ms/step - loss: 0.0718 - mean_absolute_error: 0.0553 - val_loss: 0.0611 - val_mean_absolute_error: 0.0463
Epoch 6/100
106/106 [==============================] - 9s 83ms/step - loss: 0.0672 - mean_absolute_error: 0.0515 - val_loss: 0.0597 - val_mean_absolute_error: 0.0449
Epoc

In [58]:
eval_result = model.predict(df_data['text_test'])
len(eval_result)

33886

In [55]:
df_data['y_test'].shape

(33886,)

In [60]:
from sklearn.metrics import mean_absolute_error

y_real = df_data['scaler'].inverse_transform(df_data['y_test'].reshape(-1, 1))
y_predict = df_data['scaler'].inverse_transform(eval_result.reshape(-1, 1))

mean_absolute_error(y_real, y_predict)

0.8901921765184988

In [62]:
model.save_weights('./SequenceModelWeights.h5', overwrite=True)